# import libraries

In [1]:
import sys
import os
os.chdir('E:\ISB work stuff\Post-Doc_BaligaLab_GBMSYGNALProject')
import miner as miner
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
import time
import imp
imp.reload(miner)
import miner_py3_kk as miner3

# Create directories to save output file

In [2]:
# Path to the miner directory
input_path = os.getcwd()

# create name for results folder where output files will be saved
resultsFolder = "results_miner3_gbm_MicroarrayRNA_survival_08252020"

# name of the folder containing the miner network
#networkFolder = "miner_network_results"
networkFolder = "results_minCorrelation_0o2_50_allFiles_gbm_MicroarrayRNA_survival_08252020"

# create results directory
resultsDirectory = os.path.join(os.getcwd(),resultsFolder)
if not os.path.isdir(resultsDirectory):
    os.mkdir(resultsDirectory)

expressionData=pd.read_csv("GbmMicroRNAMergedWithIDsZScored.csv",header=0,index_col=0,sep=",")


# Load primary data

In [3]:
regulonModules = miner.read_json(os.path.join(input_path,"GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\regulons.json"))

# Load primary survival data

In [5]:
# load primary survival data (i.e., corresponding to expression data training set)
survivalMMRF = pd.read_csv(os.path.join(input_path,"TCGA_Survival_Gbm.csv"),index_col=0,header=0)
survivalDfMMRF = survivalMMRF.iloc[:,0:2]
survivalDfMMRF.columns = ["duration","observed"]
kmDfMMRF = miner.kmAnalysis(survivalDf=survivalDfMMRF,durationCol="duration",statusCol="observed")
guanSurvivalDfMMRF= miner.guanRank(kmSurvival=kmDfMMRF)

# Survival analysis of regulons

In [7]:
t1 = time.time()

cox_regulons_output = miner.parallelMedianSurvivalAnalysis(regulonModules,expressionData,guanSurvivalDfMMRF,numCores=5)
cox_regulons_output = cox_regulons_output.iloc[np.argsort(np.array(cox_regulons_output.index).astype(int))]
cox_regulons_output.to_csv(os.path.join(resultsDirectory,'CoxProportionalHazardsRegulons.csv'))
cox_regulons_output.sort_values(by="HR",ascending=False,inplace=True)

print("\nHigh-risk regulons:")
print(cox_regulons_output.iloc[0:5,:])
print("\nLow-risk regulons")
print(cox_regulons_output.iloc[-5:,:])

t2 = time.time()
print("Completed regulon risk analysis in {:.2f} minutes".format((t2-t1)/60.))


High-risk regulons:
            HR       p-value
1518  5.014802  5.308803e-07
3123  4.567590  4.933645e-06
2604  4.452387  8.492082e-06
66    4.176004  2.966745e-05
116   4.162867  3.142767e-05

Low-risk regulons
            HR   p-value
1519 -3.855115  0.000116
1502 -3.877505  0.000106
195  -3.881185  0.000104
3082 -3.937179  0.000082
1545 -4.001801  0.000063
Completed regulon risk analysis in 0.82 minutes
